# 02. 토치텍스트 튜토리얼(Torchtext tutorial) - 영어
---

파이토치(PyTorch)에서는 텍스트에 대한 여러 추상화 기능을 제공하는 자연어 처리 라이브러리 토치텍스트(Torchtext)를 제공한다.  

**torchtext가 제공하는 기능**  
1. 파일 로드하기(File Loading) : 다양한 포맷의 코퍼스를 로드합니다.
2. 토큰화(Tokenization) : 문장을 단어 단위로 분리해줍니다.
3. 단어 집합(Vocab) : 단어 집합을 만듭니다.
4. 정수 인코딩(Integer encoding) : 전체 코퍼스의 단어들을 각각의 고유한 정수로 맵핑합니다.
5. 단어 벡터(Word Vector) : 단어 집합의 단어들에 고유한 임베딩 벡터를 만들어줍니다.  
   랜덤값으로 초기화한 값일 수도 있고, 사전 훈련된 임베딩 벡터들을 로드할 수도 있습니다.
6. 배치화(Batching) : 훈련 샘플들의 배치를 만들어줍니다. 이 과정에서 패딩 작업(Padding)도 이루어집니다.

## 1. 훈련 데이터와 테스트 데이터로 분리하기
---

In [4]:
# IMDB 리뷰 데이터를 다운받고 training/test로 분리하고 csv로 저장하여 사용하자 
import urllib.request
import pandas as pd

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x26dd86b9340>)

In [6]:
# 다운한 데이터를 df에 저장하고 상위 5개만 출력
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [7]:
# 전체 샘플의 개수 확인
print('전체 샘플 개수: {}'.format(len(df)))

전체 샘플 개수: 50000


In [8]:
# 5:5로 분리하여 데이터를 나누자
train_df = df[:25000]
test_df = df[25000:]

In [9]:
# 각각의 데이터셋을 csv로 저장
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

## 2. 필드 정의하기(torchtext.data)
---

In [24]:
# torchtext.data에 field라는 도구를 사용하여 앞으로 어떤 전처리를 할 것인지 정의하자
from torchtext.legacy import data

# 필드 정의
# 실제 텍스트를 위한 text 객체
TEXT = data.Field(sequential = True,      # 시퀀스 여부(default: True)
                  use_vocab = True,       # 단어 집합을 만들 것인지 여부(default: True)
                  tokenize = str.split,   # 어떤 토큰화 함수를 사용할 것인지 지정(default: string.split)
                  lower = True,           # 영어 데이터를 모두 소문자화 한다(default: False)
                  batch_first = True,     # 미니배치차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부(default: False)
                                          # True면 tensor의 맨 앞이 batch 차원이 되도록 한다.
                  fix_length = 20)        # 최대 허용 길이. 이 기준으로 padding이 된다.

# 레이블 데이터를 위한 lable 객체
LABEL = data.Field(sequential = False,
                  use_vocab = False,
                  batch_first = False,
                  is_target = True)       # 레이블 데이터 여부(defalut: False)

## 3. 데이터셋 만들기
---

In [25]:
from torchtext.legacy.data import TabularDataset

In [26]:
# field로 토큰화 수행
train_data, test_data = TabularDataset.splits(
    path='.',
    train = 'train_data.csv',
    test = 'test_data.csv',
    format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)], 
    skip_header=True    # 데이터의 첫번째 줄은 무시
)

In [28]:
# 잘 진행되었는지 확인
print('훈련 샘플의 개수:{}'.format(len(train_data)))
print('테스트 샘플의 개수:{}'.format(len(test_data)))

훈련 샘플의 개수:25000
테스트 샘플의 개수:25000


In [29]:
# vars()를 이용해 주어진 인텍스의 샘플을 확인 가능

print(vars(train_data[0]))

{'text': ['my', 'family', 'and', 'i', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>the', 'trailer', 'of', '"nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'the', 'movie', 'exceeded', 'our', 'expectations.', 'the', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'jericho', 'rosales', 'was', 'really', 'very', 'good,', "so's", 'claudine', 'barretto.', 'the', 'fact', 'that', 'i', 'despised', 'diether', 'ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'i', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'imagine', 'a', 'cynic', 'like', 'me

In [30]:
# 필드 구성 확인
print(train_data.fields.items())

dict_items([('text', <torchtext.legacy.data.field.Field object at 0x0000026E2E918550>), ('label', <torchtext.legacy.data.field.Field object at 0x0000026E2E918B50>)])


## 4. 단어 집합(Vocabulary) 만들기
---

In [31]:
# 토큰화 전처리를 끝냈으니
# 정수 인코딩(interger encoding) 작업을 진행하자
# 그를 위해선 단어 집합을 먼저 만들어야 한다

In [32]:
# 정의한 field에 .build_vocab() 도구를 사용하면 vocab을 생성할 수 있다
TEXT.build_vocab(train_data,        # 해당 데이터
                 min_freq=10,       # 단어 집합에 추가 시 단어의 최소 등장 빈도 조건
                 max_size=10000)    # 단어 집합의 최대 크기

In [33]:
print('단어 집합의 크기: {}'.format(len(TEXT.vocab)))

단어 집합의 크기: 10002


In [34]:
# 토치텍스트가 임의로 특별 토큰인 <unk>와 <pad>를 추가하였기 때문에 길이가 max_size + 2이다.
# <unk>의 번호는 0번, <pad>의 번호는 1번을 부여

In [ ]:
# 생성된 단어 집합 내의 단어들은 .stoi를 통해서 확인 가능
print(TEXT.vocab.stoi)

## 5. 토치텍스트의 데이터로더 만들기
---

In [36]:
# DataLoader는 dataset에서 mini batch만큼 데이터를 불러오는 역할을 한다. 
# torchtext 에서는 Iterator를 사용해 DataLoader를 만든다

In [37]:
from torchtext.legacy.data import Iterator

In [38]:
# 임의로 배치 크기를 아주 작은 5로 지정
batch_size = 5

In [39]:
train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

In [42]:
print('예상 미니 배치 수: {}'.format(25000/5))
print('훈련 데이터의 미니 배치 수: {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수: {}'.format(len(test_loader)))

예상 미니 배치 수: 5000.0
훈련 데이터의 미니 배치 수: 5000
테스트 데이터의 미니 배치 수: 5000


In [43]:
# 이제 첫번째 미니 배치를 받아와서 batch 라는 변수에 저장해보자
batch = next(iter(train_loader))    # iter()로 받은 train_loader의 첫번째 미니배치

In [44]:
print(type(batch))
# 타입이 tensor가 아님을 확인

<class 'torchtext.legacy.data.batch.Batch'>


In [46]:
# 실제 데이터 텐서에 접근하기 위해서는 정의한 필드명이 필요

# 첫번째 미니 배치의 text 필드를 호출해보자
print(batch.text)

tensor([[   9,   39, 1423,   63,    2,  320, 1012,    5,   10,   20,  136,   19,
         3951,    8,    0,  106,   11,   41,   71,  685],
        [  10,    7,    2,   77,    5,    3,  239,   35,  358,   63,    6,   66,
            3,  274,    4,   11,  567,   47,    6,   28],
        [2060,  147,  280,   11,   61,   91,   28,    3, 4543,  440,  440,  440,
           44,  658,    5,  116,   36, 3171,   29,    2],
        [   3,  167,    5,   89,   83,   98,    0, 5806,   25,    7,   36,   12,
          524,   18,    9,  205, 1057,    9,   98,   11],
        [   3, 1740,  405,   17,  503,  181,   12,  300,   99, 2998,    2,  650,
           40,    2,  228, 4871,   60,  583,   79,  261]])


In [ ]:
# 배치 크기가 5이기 때문에, 5개의 샘플이 출력
# 각 샘플의 길이는 20이며
# 다시 말해 하나의 미니 배치 크기는 (5 * 20)임

## 6. <pad>토큰이 사용되는 경우
---

In [ ]:
# 맨 처음 feild를 정의할 때 길이를 20이 아닌 150으로 하면 padding이 0으로 채워지는 것을 확인 할 수 있음